In [10]:
#import numpy as np#这个在某些情况下会引起bug，很奇怪
import keras
from keras.models import Model
from keras.layers import Dense
from keras.layers.pooling import GlobalAveragePooling2D
#from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.applications.vgg16 import VGG16
#from keras.applications.xception import Xception
#from keras.applications.resnet50 import ResNet50
#from keras.applications.inception_v3 import InceptionV3

print("Keras {}".format(keras.__version__))

Using TensorFlow backend.


Keras 2.0.4


In [11]:
IMG_WIDTH = 244
IMG_HEIGHT = 244
IMG_CHANNEL = 3
BATCH_SIZE = 128

# rescale=1.0/255.0这个并没有带来更好的效果，更快的收敛
train_gen = ImageDataGenerator(data_format='channels_last').flow_from_directory(
    './train/', target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True)
val_gen = ImageDataGenerator(data_format='channels_last').flow_from_directory(
    './val/', target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False)

Found 17778 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [12]:
model_vgg16_base = VGG16(input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL), weights='imagenet', include_top=False)
output = model_vgg16_base.output
## GlobalAveragePooling2D,通过Summary，它会在中间某几个维度平均，并且缩小数据规模；而flatten仅仅是把数据展开
output = GlobalAveragePooling2D()(output)
model_vgg16_pred = Dense(2, activation='softmax')(output)
model_vgg16_model = Model(inputs=model_vgg16_base.input, outputs=model_vgg16_pred)

for layer in model_vgg16_base.layers:
    # 如果为True，不仅违背了Transfer Learning的思想，而且内存小，根本算不过来
    layer.trainable = False

#model_vgg16_model.summary()

#binary_crossentropy有更好的效果，更快的收敛
#model_vgg16_model.compile(loss='categorical_crossentropy', optimizer='nadam',  metrics=['accuracy'])
model_vgg16_model.compile(loss='binary_crossentropy', optimizer='nadam',  metrics=['binary_accuracy'])

In [ ]:
# 可以跳过这步直接加载训练好的模型
LOG_DIR = './model/1'

# 既然最后衡量标准是loss，则密切关注loss，因为accuracy很有可能很快达到100%，那么loss还可以更进一步地让算法更精确
callbacks = [
    TensorBoard(log_dir=LOG_DIR, write_images=True),
#    ModelCheckpoint(LOG_DIR + "/checkpoint-{epoch:02d}-{val_acc:.2f}.hdf5", 
#                    monitor='val_acc', verbose=1, save_best_only=True),
    ModelCheckpoint(LOG_DIR + "/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5", 
                    monitor='val_loss', verbose=1, save_best_only=True),
#    EarlyStopping(monitor='val_acc', patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, verbose=1)
]
# steps_per_epoch是一个epoch中有多少step。validation_steps没多少step来validate一下
model_vgg16_model.fit_generator(train_gen, steps_per_epoch=128,
                                validation_data=val_gen, validation_steps=32,
                                callbacks=callbacks, epochs=16, verbose=1)

In [13]:
# 加载已经训练好的模型
model_vgg16_model.load_weights('./checkpoint-01-0.00.hdf5')
model_vgg16_model.evaluate_generator(val_gen, steps=1)

In [15]:
test_gen = ImageDataGenerator(data_format='channels_last').flow_from_directory(
    './test/', target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=125,
    class_mode=None, shuffle=False)

# 这里的steps 和 test_gen = ImageDataGenerator 里的 batch_size=125 乘起来要等于总数。
pred = model_vgg16_model.predict_generator(test_gen, steps=100, verbose=1)

Found 12500 images belonging to 1 classes.


In [ ]:
import re
import pandas as pd

test_ids = []
pattern = re.compile(r'\/(\d+)\.')

for i, name in enumerate(test_gen.filenames):
    test_id = pattern.findall(name)
    test_ids.append(test_id[0])

submission = pd.DataFrame({
    "id": test_ids,
    "label": pred[:,1] # dog's probability
})
submission.to_csv('prediction.csv', index=False)